In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
#!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [3]:
import pandas as pd
import numpy as np
import json

from geopy.geocoders import Nominatim
import geocoder

import requests
from bs4 import BeautifulSoup

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium
print("Libraries imported.")

Libraries imported.


In [4]:
data=requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Indore').text

In [5]:
soup = BeautifulSoup(data, 'html.parser')

In [6]:
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
idf = pd.DataFrame({"Neighborhood": neighborhoodList})

idf

,Neighborhood
0,"Azad Nagar, Indore"
1,Bhanwar Kua
2,Bijalpur
3,"Chandan Nagar, Indore"
4,"Chandra Nagar, Indore"
5,Dewas Naka
6,Geeta Bhawan
7,Indore G.P.O.
8,Khandwa Naka
9,L.I.G. Colony


In [8]:
#import geocoder
def get_latlng(neighborhood):
    coords = None
    while(coords is None):
        g = geocoder.arcgis('{}, Indore, India'.format(neighborhood))
        coords = g.latlng
    return coords
    



In [9]:
coords = [ get_latlng(neighborhood) for neighborhood in idf["Neighborhood"].tolist() ]

In [10]:
coords

[[22.69658000000004, 75.88767000000007],
 [22.687640000000044, 75.86291000000006],
 [22.66324000000003, 75.84332000000006],
 [22.709380000000067, 75.82394000000005],
 [22.74147000000005, 75.89447000000007],
 [22.771764333333323, 75.90165866666665],
 [22.71822000000003, 75.88766000000004],
 [24.977950000000078, 77.85809000000006],
 [22.689258247414326, 75.87025921921465],
 [22.734780000000057, 75.88485000000003],
 [22.741695000000004, 75.89988900000002],
 [22.70479000000006, 75.87657000000007],
 [22.727690000000052, 75.88578000000007],
 [22.67416000000003, 75.84224000000006],
 [22.734780000000057, 75.88485000000003],
 [22.755351114631523, 75.89687741798681],
 [22.76022000000006, 75.87815000000006],
 [22.756410000000074, 75.89247000000006]]

In [11]:

# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])


In [12]:
idf['Latitude'] = df_coords['Latitude']
idf['Longitude'] = df_coords['Longitude']

In [13]:
print(idf.shape)
idf

(18, 3)


,Neighborhood,Latitude,Longitude
0,"Azad Nagar, Indore",22.696580,75.887670
1,Bhanwar Kua,22.687640,75.862910
2,Bijalpur,22.663240,75.843320
3,"Chandan Nagar, Indore",22.709380,75.823940
4,"Chandra Nagar, Indore",22.741470,75.894470
5,Dewas Naka,22.771764,75.901659
6,Geeta Bhawan,22.718220,75.887660
7,Indore G.P.O.,24.977950,77.858090
8,Khandwa Naka,22.689258,75.870259
9,L.I.G. Colony,22.734780,75.884850


In [14]:
idf.to_csv("idf.csv", index=False)

In [15]:
geolocator = Nominatim(user_agent="i_explorer")
city ="Indore"
country ="India"
loc = geolocator.geocode(city+','+ country)
latitude = loc.latitude
longitude = loc.longitude
print("latitude is :-" ,latitude,"\nlongtitude is:-" ,longitude)
map_indore = folium.Map([latitude, longitude], zoom_start=10)
map_indore

latitude is :- 22.7203616 
longtitude is:- 75.8681996


In [16]:
for lat, lng, neighborhood in zip(idf['Latitude'], idf['Longitude'], idf['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_indore)  
    
map_indore

In [17]:
map_indore.save('map_indore.html')

In [18]:
CLIENT_ID = 'OXJXVBTIJOUXCAEN3W4IYSCFSNC2EBWEXXAEEV1UWEWHYTGM' 
CLIENT_SECRET = 'WRCMCR230EYROYJ0VL4HZCAMMMEHMCGW42PNDBUUORDWUXXK'
VERSION = '20180605'

In [19]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(idf['Latitude'], idf['Longitude'], idf['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VLatitude', 'VLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df

(512, 7)


,Neighborhood,Latitude,Longitude,VenueName,VLatitude,VLongitude,VenueCategory
0,"Azad Nagar, Indore",22.69658,75.88767,Cafe Coffee Day,22.697545,75.877207,Coffee Shop
1,"Azad Nagar, Indore",22.69658,75.88767,Domino's Pizza,22.696000,75.871000,Pizza Place
2,"Azad Nagar, Indore",22.69658,75.88767,Navlakha,22.702007,75.876761,Bus Station
3,"Azad Nagar, Indore",22.69658,75.88767,Hans Travels,22.702934,75.876574,Bus Station
4,"Azad Nagar, Indore",22.69658,75.88767,Nehru Stadium,22.711355,75.883097,Cricket Ground
5,"Azad Nagar, Indore",22.69658,75.88767,Shivaji Statue,22.711669,75.883256,Plaza
6,"Azad Nagar, Indore",22.69658,75.88767,ICH,22.687006,75.872435,Café
7,Bhanwar Kua,22.68764,75.86291,Regional Park,22.679106,75.855878,Park
8,Bhanwar Kua,22.68764,75.86291,Bablu Sandwich,22.703599,75.855355,Sandwich Place
9,Bhanwar Kua,22.68764,75.86291,Cafe Coffee Day,22.697545,75.877207,Coffee Shop


In [38]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VLatitude,VLongitude,VenueCategory
Neighborhood,,,,,,
"Azad Nagar, Indore",7,7,7,7,7,7
Bhanwar Kua,9,9,9,9,9,9
Bijalpur,9,9,9,9,9,9
"Chandan Nagar, Indore",5,5,5,5,5,5
"Chandra Nagar, Indore",57,57,57,57,57,57
Dewas Naka,4,4,4,4,4,4
Geeta Bhawan,53,53,53,53,53,53
Khandwa Naka,9,9,9,9,9,9
L.I.G. Colony,68,68,68,68,68,68


In [25]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 46 uniques categories.


In [26]:
 venues_df['VenueCategory'].unique()

array(['Coffee Shop', 'Pizza Place', 'Bus Station', 'Cricket Ground',
       'Plaza', 'Café', 'Park', 'Sandwich Place', 'Lounge', 'Tea Room',
       'Multiplex', 'Train Station', 'Diner', 'Gym', 'Discount Store',
       'Indian Restaurant', 'ATM', 'Grocery Store', 'Mattress Store',
       'Golf Course', 'Hookah Bar', 'Shopping Mall', 'Pub',
       'Chinese Restaurant', 'Hotel', 'Mediterranean Restaurant',
       'Fast Food Restaurant', 'Dessert Shop', 'BBQ Joint', 'Market',
       'Bakery', 'Gym / Fitness Center', 'Restaurant', 'Burger Joint',
       'IT Services', 'Movie Theater', 'Cafeteria', 'Ice Cream Shop',
       'Hot Dog Joint', 'Snack Place', 'Italian Restaurant',
       'Clothing Store', 'Spa', 'Arts & Entertainment', 'Breakfast Spot',
       'Airport'], dtype=object)

In [39]:
"Neighborhood" in venues_df['VenueCategory'].unique()

False

In [40]:
ionehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ionehot['Neighborhood'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ionehot.columns[-1]] + list(ionehot.columns[:-1])
ionehot = ionehot[fixed_columns]

print(ionehot.shape)
ionehot.head()

(512, 47)


,Neighborhood,ATM,Airport,Arts & Entertainment,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Bus Station,Cafeteria,...,Pizza Place,Plaza,Pub,Restaurant,Sandwich Place,Shopping Mall,Snack Place,Spa,Tea Room,Train Station
0,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,"Azad Nagar, Indore",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,"Azad Nagar, Indore",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
ionehot['rest']=ionehot['Mediterranean Restaurant']+ionehot['Restaurant']+ionehot['Indian Restaurant']+ionehot['Chinese Restaurant']+ionehot['Fast Food Restaurant']+ionehot['Italian Restaurant']
ionehot

,Neighborhood,ATM,Airport,Arts & Entertainment,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Bus Station,Cafeteria,...,Plaza,Pub,Restaurant,Sandwich Place,Shopping Mall,Snack Place,Spa,Tea Room,Train Station,rest
0,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Azad Nagar, Indore",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,"Azad Nagar, Indore",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
6,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Bhanwar Kua,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Bhanwar Kua,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,Bhanwar Kua,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
i_grouped = ionehot.groupby(["Neighborhood"]).mean().reset_index()

print(i_grouped.shape)
i_grouped

(17, 48)


,Neighborhood,ATM,Airport,Arts & Entertainment,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Bus Station,Cafeteria,...,Plaza,Pub,Restaurant,Sandwich Place,Shopping Mall,Snack Place,Spa,Tea Room,Train Station,rest
0,"Azad Nagar, Indore",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.285714,0.000000,...,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bhanwar Kua,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000
2,Bijalpur,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.222222,0.222222
3,"Chandan Nagar, Indore",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Chandra Nagar, Indore",0.000000,0.000000,0.000000,0.017544,0.052632,0.000000,0.017544,0.000000,0.000000,...,0.000000,0.017544,0.035088,0.035088,0.052632,0.000000,0.000000,0.000000,0.000000,0.298246
5,Dewas Naka,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Geeta Bhawan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018868,...,0.056604,0.000000,0.018868,0.018868,0.018868,0.037736,0.018868,0.018868,0.000000,0.207547
7,Khandwa Naka,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.222222,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000
8,L.I.G. Colony,0.000000,0.000000,0.000000,0.014706,0.029412,0.000000,0.014706,0.000000,0.014706,...,0.029412,0.014706,0.000000,0.044118,0.058824,0.014706,0.000000,0.014706,0.000000,0.250000
9,"Malviya Nagar, Indore",0.000000,0.000000,0.000000,0.020408,0.061224,0.000000,0.020408,0.000000,0.000000,...,0.000000,0.020408,0.020408,0.040816,0.061224,0.000000,0.000000,0.000000,0.000000,0.244898


In [44]:
#Restaurant=venues_df.(['Chinese Restaurant','Restaurant','Indian Restaurant','Mediterranean Restaurant', 'Fast Food Restaurant','Italian Restaurant','Greek Restaurant'], axis=1)

In [46]:
len(i_grouped[i_grouped["rest"] > 0])

12

In [48]:
kl_mall = i_grouped[["Neighborhood","rest"]]
kl_mall

,Neighborhood,rest
0,"Azad Nagar, Indore",0.000000
1,Bhanwar Kua,0.000000
2,Bijalpur,0.222222
3,"Chandan Nagar, Indore",0.000000
4,"Chandra Nagar, Indore",0.298246
5,Dewas Naka,0.000000
6,Geeta Bhawan,0.207547
7,Khandwa Naka,0.000000
8,L.I.G. Colony,0.250000
9,"Malviya Nagar, Indore",0.244898


In [49]:
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 1, 2, 1, 0, 1, 0, 0], dtype=int32)

In [50]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [52]:

kl_merged.rename(columns={"rest": "Restaurant"}, inplace=True)
kl_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,"Azad Nagar, Indore",0.000000,1
1,Bhanwar Kua,0.000000,1
2,Bijalpur,0.222222,0
3,"Chandan Nagar, Indore",0.000000,1
4,"Chandra Nagar, Indore",0.298246,2


In [54]:
kl_merged = kl_merged.join(idf.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head()

(17, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,"Azad Nagar, Indore",0.000000,1,22.69658,75.88767
1,Bhanwar Kua,0.000000,1,22.68764,75.86291
2,Bijalpur,0.222222,0,22.66324,75.84332
3,"Chandan Nagar, Indore",0.000000,1,22.70938,75.82394
4,"Chandra Nagar, Indore",0.298246,2,22.74147,75.89447


In [55]:
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(17, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
8,L.I.G. Colony,0.250000,0,22.734780,75.884850
14,Satya Sai Chouraha,0.257143,0,22.755351,75.896877
13,Rau Colony,0.250000,0,22.734780,75.884850
12,"Rajendra Nagar, Indore",0.142857,0,22.674160,75.842240
11,Palasia,0.224138,0,22.727690,75.885780
10,Navlakha,0.193548,0,22.704790,75.876570
9,"Malviya Nagar, Indore",0.244898,0,22.741695,75.899889
16,"Vijay Nagar, Indore",0.257143,0,22.756410,75.892470
6,Geeta Bhawan,0.207547,0,22.718220,75.887660
2,Bijalpur,0.222222,0,22.663240,75.843320


In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
map_clusters.save('map_clusters.html')

In [58]:

kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
8,L.I.G. Colony,0.250000,0,22.734780,75.884850
14,Satya Sai Chouraha,0.257143,0,22.755351,75.896877
13,Rau Colony,0.250000,0,22.734780,75.884850
12,"Rajendra Nagar, Indore",0.142857,0,22.674160,75.842240
11,Palasia,0.224138,0,22.727690,75.885780
10,Navlakha,0.193548,0,22.704790,75.876570
9,"Malviya Nagar, Indore",0.244898,0,22.741695,75.899889
16,"Vijay Nagar, Indore",0.257143,0,22.756410,75.892470
6,Geeta Bhawan,0.207547,0,22.718220,75.887660
2,Bijalpur,0.222222,0,22.663240,75.843320


In [59]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
5,Dewas Naka,0.0,1,22.771764,75.901659
3,"Chandan Nagar, Indore",0.0,1,22.709380,75.823940
1,Bhanwar Kua,0.0,1,22.687640,75.862910
7,Khandwa Naka,0.0,1,22.689258,75.870259
0,"Azad Nagar, Indore",0.0,1,22.696580,75.887670


In [60]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
15,"Sukhliya, Indore",0.375000,2,22.76022,75.87815
4,"Chandra Nagar, Indore",0.298246,2,22.74147,75.89447
